In [32]:
import numpy as np


class LogisticRegression():

    def __init__(self, lr=1e-4, iterations=1000, l2_penalty=0):
        
        self.iterations = iterations
        self.l2_penalty = l2_penalty
        self.lr = lr
        self.parameters = []

    def _init_params(self, n_features):

        self.parameters = np.random.normal(size=(n_features + 1), scale=0.1) #initalize n weights + 1 bias randomly
        self.parameters = self.parameters.reshape(-1, 1) # make it matrix

    @staticmethod
    def _get_design_matrix(X):
        n, d = X.shape
        X_d = np.empty((n, d+1), dtype=X.dtype)
        X_d[:, 0] = 1 #column of ones
        X_d[:, 1:] = X
        return X_d

    @staticmethod
    def _sigmoid(logits):
        return 1 / (1 + np.exp(-logits))

    def _compute_logits(self, X):        
        return (X @ self.parameters)  #(m, n) x (n, 1) =  (m, 1) #logits
    
    @staticmethod
    def _binary_cross_entropy_loss(Y_true, Y_pred):
        
        loss = - (Y_true * np.log(Y_pred) + (1 - Y_true) * np.log(1 - Y_pred))
        return loss.mean()
    
    def _compute_gradient(self, X_d, Y_true, Y_pred):
        """
        X_d:    (m × (d+1)) design matrix, X_d[:,0] == 1
        Y_true: (m × 1) true labels
        Y_pred: (m × 1) predicted probabilities
        """
        
        m = Y_true.shape[0]
        gradient = X_d.T @ (Y_pred - Y_true)  / m  #  (n, m)  @ (m, 1) = (n, 1) #gradient

        #l2 regularization
        gradient[1:] += (self.parameters[1:] * self.l2_penalty) / m #add sum of weights (except bias term)

        return gradient
    
    def fit(self, X, Y):

        if not (isinstance(X, np.ndarray) and isinstance(Y, (np.ndarray, np.array, list))):
            raise ValueError("Invalid type for inputs. Expected numpy.ndarray")
        if len(Y.shape) != 2:
            raise ValueError("Y must be matrix. Hint: do Y.reshape(-1, 1)")
        
        self.n_features = X.shape[1]
        self._init_params(self.n_features)

        #create design matrix
        X_d  = self._get_design_matrix(X)

        #Training loop
        for i in range(self.iterations):
            
            logits = self._compute_logits(X_d)
            Y_pred = self._sigmoid(logits)

            #compute loss
            loss = self._binary_cross_entropy_loss(Y, Y_pred)
            print(f"ITERATION {i + 1} | BCE LOSS = {float(loss.round(6))}")

            #compute gradient
            gradient = self._compute_gradient(X_d, Y, Y_pred)

            #update parameters
            self.parameters -= self.lr * gradient






In [33]:
logreg = LogisticRegression(lr=0.01, iterations=10000)

X = np.random.normal(size=(50, 4))
y = np.ones((50, 1))
y[5:15] = 0
X[3, 5:15] += 6  


logreg.fit(X, y)

ITERATION 1 | BCE LOSS = 0.755392
ITERATION 2 | BCE LOSS = 0.754021
ITERATION 3 | BCE LOSS = 0.752658
ITERATION 4 | BCE LOSS = 0.751302
ITERATION 5 | BCE LOSS = 0.749954
ITERATION 6 | BCE LOSS = 0.748613
ITERATION 7 | BCE LOSS = 0.747279
ITERATION 8 | BCE LOSS = 0.745952
ITERATION 9 | BCE LOSS = 0.744633
ITERATION 10 | BCE LOSS = 0.743321
ITERATION 11 | BCE LOSS = 0.742015
ITERATION 12 | BCE LOSS = 0.740717
ITERATION 13 | BCE LOSS = 0.739426
ITERATION 14 | BCE LOSS = 0.738142
ITERATION 15 | BCE LOSS = 0.736865
ITERATION 16 | BCE LOSS = 0.735595
ITERATION 17 | BCE LOSS = 0.734332
ITERATION 18 | BCE LOSS = 0.733075
ITERATION 19 | BCE LOSS = 0.731826
ITERATION 20 | BCE LOSS = 0.730583
ITERATION 21 | BCE LOSS = 0.729347
ITERATION 22 | BCE LOSS = 0.728118
ITERATION 23 | BCE LOSS = 0.726895
ITERATION 24 | BCE LOSS = 0.72568
ITERATION 25 | BCE LOSS = 0.72447
ITERATION 26 | BCE LOSS = 0.723268
ITERATION 27 | BCE LOSS = 0.722072
ITERATION 28 | BCE LOSS = 0.720882
ITERATION 29 | BCE LOSS = 0.719